<font color=#010190 size = 8> Deployment

<font color=#010190 size = 5> Libraries Import

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from flask import Flask, flash, request, redirect, url_for, render_template, session
from flaskext.mysql import MySQL
import pymysql
import re
import urllib.request
import os
from werkzeug.utils import secure_filename
import scipy.misc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
from skimage.io import imread, imshow
import pytesseract
from os import listdir
from os.path import isfile, join
import numpy
import cv2
import argparse
from PIL import Image
import glob 
from cv2 import cv2
import pytesseract
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import pickle

<font color=#010190 size = 5> Initialising Flask App

In [2]:
app = Flask(__name__)

<font color=#010190 size = 5> Car Price prediction Model Deployment

In [3]:
model = pickle.load(open('rf_random', 'rb')) # load ml model
sc = pickle.load(open('scc', 'rb')) # load ml model
ex = pickle.load(open('EX', 'rb')) # load ml model
assupred = pickle.load(open('GBT', 'rb')) # load ml model
assutype = pickle.load(open('knn', 'rb')) # load ml model
std = pickle.load(open('std', 'rb')) # load ml model

standardScaler = StandardScaler()
standardScaler.fit(sc)


@app.route('/carpred', methods=['GET'])
def home():
    return render_template('car_pred.html')


@app.route('/carpred', methods=['POST', 'GET'])
def predict():
    if request.method == 'POST':
        Kilometers_Driven = float(request.form['Kilometers_Driven'])
        Engine = float(request.form['Engine'])
        seats = float(request.form['seats'])
        car_age = int(request.form['age'])
        fuel_type = request.form['fuel']
        transmission_type = request.form['transmission']

        
        
        if fuel_type == 'Diesel':
            fuel_type = 1
        elif fuel_type == 'Petrol':
            fuel_type = 0
        elif fuel_type == 'CNG':
            fuel_type = 2
        elif fuel_type == 'LPG':
            fuel_type = 3
        else:
            fuel_type = 4
            

        if transmission_type == 'Manual':
            transmission_type = 1
        else:
            transmission_type = 0
        
        features = np.array
        ([[car_age,Kilometers_Driven,fuel_type,transmission_type,Engine,seats,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]])

        scc=0
        print(scc)
        #model = pickle.load(open('model', 'rb'))  # load ml model
        prediction = model.predict(standardScaler.transform([[car_age,Kilometers_Driven,fuel_type,transmission_type,Engine,seats,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]]))
        output = round(prediction[0]/2.7, 2)
     #   output =ex('CIN1.jpg','13CG_F.jpg','1CG_B.jpg')
        return render_template('car_pred.html', output="{} DT".format(output))

<font color=#010190 size = 5> Insurance Price prediction Model Deployment

In [4]:
@app.route('/assupred', methods=['GET'])
def homeassu():
    return render_template('assu_pred.html')
       
@app.route('/assupred', methods=['POST', 'GET'])
def assupredict():
    if request.method == 'POST':
        selling_price = float(request.form['selling_price'])
        Engine = int(request.form['Engine'])
        type_ass = request.form['type_ass']
        car_age = int(request.form['age'])  
        if type_ass == 'routiere+options':
            type_ass = 0
        elif type_ass == 'dommage et collision':
            type_ass = 1
        else: 
            type_ass = 2
            
        featuresassu = np.array
        ([[selling_price,Engine,type_ass,car_age]])

        scc=0
        print(scc)
        predictionassu = assupred.predict(std.transform([[selling_price,Engine,type_ass,car_age]]))  
        return render_template('assu_pred.html', predictionassu="{} DT".format(predictionassu))

<font color=#010190 size = 5> Insurance Type Suggestion Model Deployment

In [5]:
@app.route('/typepred', methods=['GET'])
def hometypeassu():
    return render_template('assutype.html')
    
    
@app.route('/typepred', methods=['POST', 'GET'])
def typeassupredict():
    if request.method == 'POST':
        selling_price = float(request.form['selling_price'])
        Engine = int(request.form['Engine'])
        prix_assuran = float(request.form['prix_assuran'])
        car_age = int(request.form['age'])       
        featuresassu = np.array
        ([[selling_price,Engine,prix_assuran,car_age]])
        predictiontypeassu = assutype.predict([[selling_price,Engine,prix_assuran,car_age]]) 
        return render_template('assutype.html', predictiontypeassu="{} ".format(predictiontypeassu))

<font color=#010190 size = 5> Data Extraction Model

In [6]:
def numberinstring(FP):
    num = ""
    for c in FP:
        if c.isdigit():
            num = num + c
    return num

def extraction(imS, imageFrontCG, imageBackCG):
    df = pd.DataFrame(columns=['CIN number','First Name','Last Name','Year of Birth',
                           'Body','Fiscal Power','Empty weight','Number of axies',
                           'Registration right','Registration left','Constructor','Commercial Type',
                           'Serial Number','Release','Type of constructor'])
    
    #CARTE D'IDENTITE
    #imS = cv2.resize(imS, (600,400))
    img = imS
    img_copy = imS.copy()
    img_canny = cv2.Canny(img_copy, 50, 100, apertureSize = 3)
    img_hough = cv2.HoughLinesP(img_canny, 1, math.pi / 180, 100, minLineLength = 100, maxLineGap = 10)
    (x, y, w, h) = (np.amin(img_hough, axis = 0)[0,0], np.amin(img_hough, axis = 0)[0,1], np.amax(img_hough, axis = 0)[0,0] - np.amin(img_hough, axis = 0)[0,0], np.amax(img_hough, axis = 0)[0,1] - np.amin(img_hough, axis = 0)[0,1])
    img_roi = img_copy[y:y+h,x:x+w]
    img_roi_copy = imS.copy()
    
    #NUM CIN
    dim_mrz = (x, y, w, h) = (230, 130, 180, 45)
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    img_mrz = imS[y:y+h, x:x+w]
    img_mrz =cv2.GaussianBlur(img_mrz, (3,3), 0)
    ret, img_mrz = cv2.threshold(img_mrz,127,255,cv2.THRESH_TOZERO)
    Cin_num = pytesseract.image_to_string(img_mrz, lang='eng')
    Cin_num
    characters = "\n"
    for x in range(len(characters)):
        Cin_num = Cin_num.replace(characters[x],"")
    Cin_num
    
    #Last Name
    dim_lastname_chi = (x, y, w, h) = (380, 180, 160, 60)
    img_lastname_chi = imS[y:y+h, x:x+w]
    img_lastname_chi = cv2.GaussianBlur(img_lastname_chi, (3,3), 0)
    ret, img_lastname_chi = cv2.threshold(img_lastname_chi,127,255,cv2.THRESH_TOZERO)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    LName = pytesseract.image_to_string(img_lastname_chi, lang='ara')
    characters = "\n"
    for x in range(len(characters)):
        LName = LName.replace(characters[x],"")
    LName
    
    #First Name
    dim_firstname_chi = (x, y, w, h) = (380, 240, 160, 36)
    img_firstname_chi = imS[y:y+h, x:x+w]
    img_firstname_chi = cv2.GaussianBlur(img_firstname_chi, (3,3), 0)
    ret, img_firstname_chi = cv2.threshold(img_firstname_chi,127,255,cv2.THRESH_TRUNC)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    FName = pytesseract.image_to_string(img_firstname_chi, lang='ara')
    for x in range(len(characters)):
        FName = FName.replace(characters[x],"")
    FName
    
    #Birthday
    dim_Birthdate_chi = (x, y, w, h) = (250, 310, 246, 43)
    img_Birthdate_chi = imS[y:y+h, x:x+w]
    img_Birthdate_chi = cv2.GaussianBlur(img_Birthdate_chi, (3,3), 0)
    ret, img_Birthdate_chi = cv2.threshold(img_Birthdate_chi,127,255,cv2.THRESH_TRUNC)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    YB = pytesseract.image_to_string(img_Birthdate_chi, lang='ara')
    YB
    characters = "\n"
    for x in range(len(characters)):
        YB = YB.replace(characters[x],"")
    YB
    
    #CARTE GRISE
    #imS = cv2.resize(imageFrontCG, (600,400))
    
    #Type of constructor
    dim_mrz = (x, y, w, h) = (300, 310, 130, 30)
    img_roi_copy = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_toc = img_roi_copy[y:y+h, x:x+w]
    img_toc = cv2.GaussianBlur(img_toc, (3,3), 0)
    ret, img_toc = cv2.threshold(img_toc,127,255,cv2.THRESH_TRUNC)
    TOC = pytesseract.image_to_string(img_toc, lang='eng')
    characters = "\n"
    for x in range(len(characters)):
        TOC = TOC.replace(characters[x],"")
    TOC
    
    #Contructor
    dim_constructor = (x, y, w, h) = (80, 315, 160, 30)
    img_constructor = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_constructor = img_roi_copy[y:y+h, x:x+w]
    img_constructor =cv2.GaussianBlur(img_constructor, (3,3), 0)
    ret, img_constructor = cv2.threshold(img_constructor,127,255,cv2.THRESH_TRUNC)
    constructor = pytesseract.image_to_string(img_constructor, lang='eng')
    characters = "\n"
    for x in range(len(characters)):
        constructor = constructor.replace(characters[x],"")
    constructor
    
    #Type commercial
    dim_tc = (x, y, w, h) = (73, 365, 168, 25)
    img_tc = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_tc = img_roi_copy[y:y+h, x:x+w]
    img_tc =cv2.GaussianBlur(img_tc, (3,3), 0)
    ret, img_tc = cv2.threshold(img_tc,127,255,cv2.THRESH_TRUNC)
    tc = pytesseract.image_to_string(img_tc, lang='eng')
    characters = "\n"
    for x in range(len(characters)):
        tc = tc.replace(characters[x],"")
    tc
    
    #Release
    dim_release = (x, y, w, h) = (300, 340, 130, 28)
    img_release = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_release = img_roi_copy[y:y+h, x:x+w]
    img_release =cv2.GaussianBlur(img_release, (3,3), 0)
    ret, img_release = cv2.threshold(img_release,127,255,cv2.THRESH_TRUNC)
    release = pytesseract.image_to_string(img_release, lang='eng')
    characters = "\n"
    for x in range(len(characters)):
        release = release.replace(characters[x],"")
    release
    
    #Serial number
    dim_SN = (x, y, w, h) = (80, 275, 220, 30)
    img_SN = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_SN = img_roi_copy[y:y+h, x:x+w]
    img_SN =cv2.GaussianBlur(img_SN, (3,3), 0)
    ret, img_SN = cv2.threshold(img_SN,127,255,cv2.THRESH_TRUNC)
    SN = pytesseract.image_to_string(img_SN, lang='eng') 
    characters = "\n"
    for x in range(len(characters)):
        SN = SN.replace(characters[x],"")
    SN
    
    #Registration Left
    dim_RL = (x, y, w, h) = (15, 230, 50, 170)
    img_RL = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_RL = img_roi_copy[y:y+h, x:x+w]
    img_RL =cv2.GaussianBlur(img_RL, (3,3), 0)
    ret, img_RL = cv2.threshold(img_RL,127,255,cv2.THRESH_TRUNC)
    img_RL_rot = cv2.rotate(img_RL, cv2.ROTATE_90_CLOCKWISE)
    RL = pytesseract.image_to_string(img_RL_rot, config = '--psm 12')
    characters = "\n"
    for x in range(len(characters)):
        RL = RL.replace(characters[x],"")
    RL
    
    #Registration Right
    dim_RR = (x, y, w, h) = (20, 20, 50, 160)
    img_RR = cv2.rectangle(imS, (x, y), (x + w ,y + h),(0,0,0),2)
    img_RR = img_roi_copy[y:y+h, x:x+w]
    img_RR =cv2.GaussianBlur(img_RR, (3,3), 0)
    ret, img_RL = cv2.threshold(img_RR,127,255,cv2.THRESH_TRUNC)
    img_RR_rot = cv2.rotate(img_RR, cv2.ROTATE_90_CLOCKWISE)
    RR = pytesseract.image_to_string(img_RR_rot, config = '--psm 8')
    characters = "\n"
    for x in range(len(characters)):
        RR = RR.replace(characters[x],"")
    RR = numberinstring(RR)
    RR
    
    #CARTE GRISE
    imS = cv2.resize(imageBackCG, (600,400))   
    img = imS
    img_copy = imS.copy()
    img_canny = cv2.Canny(img_copy, 50, 100, apertureSize = 3)
    img_hough = cv2.HoughLinesP(img_canny, 1, math.pi / 180, 100, minLineLength = 100, maxLineGap = 10)
    img_roi = img_copy[y:y+h,x:x+w]
    
    #Body
    dim_Doors_chi = (x, y, w, h) = (380,25, 125, 26)
    img_Doors_chi = imS[y:y+h, x:x+w]
    img_Doors_chi = cv2.GaussianBlur(img_Doors_chi, (3,3), 0)
    ret, img_Doors_chi = cv2.threshold(img_Doors_chi,127,255,cv2.THRESH_TRUNC)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    Body = pytesseract.image_to_string(img_Doors_chi, lang='ara')
    for x in range(len(characters)):
        Body = Body.replace(characters[x],"")
    Body
    
    #Cylindre
    dim_Cylindre_chi = (x, y, w, h) = (120,70, 90, 30)
    img_Cylindre_chi = imS[y:y+h, x:x+w]
    img_Cylindre_chi = cv2.GaussianBlur(img_Cylindre_chi, (3,3), 0)
    ret, img_Cylindre_chi = cv2.threshold(img_Cylindre_chi,127,255,cv2.THRESH_TRUNC)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    NA = pytesseract.image_to_string(img_Cylindre_chi, lang='ara')
    NA = numberinstring(NA)
    NA
    
    #Fiscal power
    dim_FP_chi = (x, y, w, h) = (430,67, 80, 27)
    img_FP_chi = imS[y:y+h, x:x+w]
    img_FP_chi = cv2.GaussianBlur(img_FP_chi, (3,3), 0)
    ret, img_FP_chi = cv2.threshold(img_FP_chi,127,255,cv2.THRESH_TRUNC)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    FP = pytesseract.image_to_string(img_FP_chi, config = '--psm 12')
    FP=numberinstring(FP)
    FP
    
    #EW
    dim_EW_chi = (x, y, w, h) = (420,90, 160, 27)
    img_EW_chi = imS[y:y+h, x:x+w]
    img_EW_chi = cv2.GaussianBlur(img_EW_chi, (3,3), 0)
    ret, img_EW_chi = cv2.threshold(img_EW_chi,127,255,cv2.THRESH_TRUNC)
    img_roi_copy = imS.copy()
    img_roi_copy = cv2.rectangle(img_roi_copy, (x, y), (x + w ,y + h),(0,0,0),2)
    EW = pytesseract.image_to_string(img_EW_chi, config = '--psm 12')
    EW=numberinstring(EW)
    EW
    
    #Filling the data base after extraction
    df = df.append({'CIN number':Cin_num,'First Name':FName,'Last Name':LName,'Year of Birth':YB,
                           'Body':Body,'Fiscal Power':FP,'Empty weight':EW,'Number of axies':NA,
                           'Registration right':RR,'Registration left':RL,'Constructor':tc,'Commercial Type':tc,
                           'Serial Number':SN,'Release':release,'Type of constructor':TOC}, ignore_index=True)
    df.to_excel('BD_extract.xlsx')


<font color=#010190 size = 5> Data Extraction Model Deployment

In [7]:
UPLOAD_FOLDER = 'static/uploads/'
 
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
    
#******************************************************CIN EXTRACTION ***************************************  
@app.route('/extractCIN', methods=['POST'])
def extractCIN():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        flash('Image successfully uploaded and displayed below')
        return render_template('extractCIN.html')
    else:
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)
@app.route('/extractCIN')
def extractCIN2():
    return render_template('extractCIN.html')

#******************************************************FRONT GARY CARD ***************************************  
@app.route('/extractCGF', methods=['POST'])
def extractCGF():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        flash('Image successfully uploaded and displayed below')
        return render_template('extractCGF.html')
    else:
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)
    
@app.route('/extractCGF')
def extractCGF2():
    return render_template('extractCGF.html')
#******************************************************Back GARY CARD ***************************************  
@app.route('/extractCGB', methods=['POST'])
def extractCGB():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        flash('Image successfully uploaded and displayed below')
        return render_template('extractCGB.html')
    else:
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)
@app.route('/extractCGB')
def extractCGB2():
    return render_template('extractCGB.html')
#******************************************************Final extraction ***************************************  
@app.route('/extract_final', methods=['GET','POST'])
def extract_final():
    onlyfiles = [f for f in listdir('static/uploads') if isfile(join('static/uploads', f))]
    #imS = cv2.imread('static/uploads'+onlyfiles[0])
    #imageFrontCG = cv2.imread('static/uploads'+onlyfiles[1])
    #imageBackCG = cv2.imread('static/uploads'+onlyfiles[2])
    imS = cv2.imread('static/uploads/CIN4.jpg')
    imageFrontCG = cv2.imread('static/uploads/4CG_F.jpg')
    imageBackCG = cv2.imread('static/uploads/4CG_B.jpg')
    extraction(imS, imageFrontCG, imageBackCG)
    flash('Extraction succeeded')
    
    return render_template('extract_final.html')
import pdfkit
@app.route('/test')
def test2():    
    name = "Giovanni Smith"
    html = render_template(
        "test.html",
        name=name)
    pdf = pdfkit.from_string(html, False)
    response = make_response(pdf)
    response.headers["Content-Type"] = "application/pdf"
    response.headers["Content-Disposition"] = "inline; filename=output.pdf"
    return response

<font color=#010190 size = 5>  Contract generation in PDF

In [8]:
@app.route('/printpdf')
def printpdf():
    return render_template('contract.html')

<font color=#010190 size = 5> Login

In [9]:
app.secret_key = 'cairocoders-ednalan'

mysql = MySQL()

# MySQL configurations
app.config['MYSQL_DATABASE_USER'] = 'root'
app.config['MYSQL_DATABASE_PASSWORD'] = ''
app.config['MYSQL_DATABASE_DB'] = 'pi'
app.config['MYSQL_DATABASE_HOST'] = 'localhost'
mysql.init_app(app)

# http://localhost:5000/pythonlogin/ - this will be the login page
@app.route('/pythonlogin/', methods=['GET', 'POST'])
def login():
 # connect
    conn = mysql.connect()
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # Output message if something goes wrong...
    msg = ''
    # Check if "username" and "password" POST requests exist (user submitted form)
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
        # Create variables for easy access
        username = request.form['username']
        password = request.form['password']
        # Check if account exists using MySQL
        cursor.execute('SELECT * FROM accounts WHERE username = %s AND password = %s', (username, password))
        # Fetch one record and return result
        account = cursor.fetchone()

    # If account exists in accounts table in out database
        if account:
            # Create session data, we can access this data in other routes
            session['loggedin'] = True
            session['id'] = account['id']
            session['username'] = account['username']
            # Redirect to home page
            #return 'Logged in successfully!'
            return redirect(url_for('dash'))
        else:
            # Account doesnt exist or username/password incorrect
            msg = 'Incorrect username/password!'

    return render_template('login.html', msg=msg)

@app.route('/profile')
def profile():
 # Check if account exists using MySQL
    conn = mysql.connect()
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # Check if user is loggedin
    if 'loggedin' in session:
        # We need all the account info for the user so we can display it on the profile page
        cursor.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = cursor.fetchone()
        # Show the profile page with account info
        return render_template('profile.html', account=account)
    # User is not loggedin redirect to login page
    return redirect(url_for('login'))

<font color=#010190 size = 5> Registration

In [10]:
@app.route('/register', methods=['GET', 'POST'])
def register():
 # connect
    conn = mysql.connect()
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # Output message if something goes wrong...
    msg = ''
    # Check if "username", "password" and "email" POST requests exist (user submitted form)
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form and 'email' in request.form:
        # Create variables for easy access
        fullname = request.form['fullname']
        username = request.form['username']
        password = request.form['password']
        email = request.form['email']

  #Check if account exists using MySQL
        cursor.execute('SELECT * FROM accounts WHERE username = %s', (username))
        account = cursor.fetchone()
        # If account exists show error and validation checks
        if account:
            msg = 'Account already exists!'
        elif not re.match(r'[^@]+@[^@]+\.[^@]+', email):
            msg = 'Invalid email address!'
        elif not re.match(r'[A-Za-z0-9]+', username):
            msg = 'Username must contain only characters and numbers!'
        elif not username or not password or not email:
            msg = 'Please fill out the form!'
        else:
            # Account doesnt exists and the form data is valid, now insert new account into accounts table
            cursor.execute('INSERT INTO accounts VALUES (NULL, %s, %s, %s, %s)', (fullname, username, password, email))
            conn.commit()

            msg = 'You have successfully registered!'
    elif request.method == 'POST':
        # Form is empty... (no POST data)
        msg = 'Please fill out the form!'
    # Show registration form with message (if any)
    return render_template('register.html', msg=msg)

<font color=#010190 size = 5> Log Out

In [11]:
@app.route('/logout')
def logout():
    # Remove session data, this will log the user out
   session.pop('loggedin', None)
   session.pop('id', None)
   session.pop('username', None)
   # Redirect to login page
   return redirect(url_for('login'))

<font color=#010190 size = 5> Launching the server

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
